# Content
* ### 1. [Framework](1.Framework)
* ### 2. [Environment Setup]()
* ### 3. [Data Preparation]()
* ### 4. [Launch training]()
* ### 5. [Inference & Evaluation]()

## 1. Framework


Transfer Learning Kit is a general and convenient framework for transfer knowledge from pretrained model and/or source domain data to target task. Its objectives are:
* Transfer knowledge from pretrained model with the same/different network structure, which greatly speedups training without accuracy regression.
* Transfer knowledge from source domain data without target label.

The hierarchy of Transfer Learning Kit is list below. And, there are 5 key components in our Transfer Learning Kit:

1.	Backbone Factory: creates a backbone net according to predefined backbone or user-provided backbone to make basic prediction. 
2.	Task Finetunner: creates a pretrained finetuning schema (called “finetunner”) to transfer knowledge from a pretrained model to target model with the same network structure.
3.	Domain Adapter: creates a domain adaption net (called “adapter”) to transfer knowledge from source domain to target domain.
4.	Knowledge Distiller: creates a knowledge distillation net (called “distiller”) to transfer knowledge from teacher model to target model. 
5.	Transferrable Model: creates a customized and transferrable model which is a wrapper of backbone, adapter and distiller.
![Framework](doc/imgs/framework.png)

## 2. Adapter
Transfer knowledge from source domain(cheap labels) to target domain (label-free).

* Direct applying pre-trained model into target domain always cannot work due to covariate shift and label shift,  while labeling could be expensive in some domains and delays the model deployment time, which make fine-tuning not working.
* Adapter aims at reusing the transferable knowledge with the help of another labeled dataset with same learning task. That is, achieving better generalization with little labeled target dataset or achieving a competitive performance in label-free target dataset.
![Adapter](doc/imgs/adapter.png)

## 3. Try it

### 3.1 Task Description
* In this demo, we will introduce how to use domain adaptation to transfer knowledge in medical image semantic segmentation
* Our source domain is AMOS dataset(Download AMOS data from [here](https://amos22.grand-challenge.org/Dataset/)), which provides 500 CT and 100 MRI scans with voxel-level annotations of 15 abdominal organs, including the spleen, right kidney, left kidney, gallbladder, esophagus, liver, stomach, aorta, inferior vena cava, pancreas, right adrenal gland, left adrenal gland, duodenum, bladder, prostate/uterus.
* Our target domain is KiTS dataset(Download KiTS data from [here](https://github.com/neheller/kits19)), which provides 300 CT scans with voxel-level annotations of kidney organs and kidney tumor.
* Our task is to explore reliable kidney semantic segmentation methodologies with the help of labeled AMOS dataset and unlabeled KiTS dataset, evalutaion metric is kidney dice score in target domain.
* We can see from the following picture, **even without the target label data, adapter achieve 10.67x training speedup, while keep the 93% performance ratio.**

![adapter_result_plot](doc/imgs/adapter_result_plot.png)

### 3.2 domain adaptation from AMOS to KiTS
- We will first pre-train model in AMOS dataset, and use this pre-trained model later for prameter initialization for domain adaptation
- We use [3D-UNet](https://arxiv.org/abs/1606.06650) to train the model
- Now we apply domain adaptation algorithm to transfer knowledge from AMOS dataset to KiTS dataset
- We use a DANN-like model architecture, the DANN algorithm is illustrated as follows:
![dann](doc/imgs/dann.png)
- Notice: 
    - we donot use **any label** from target domain KiTS, we only use label from source domain AMOS for training
    - *For demostration, we only train 1 epochs:*

In [9]:
!cd cd /home/vmagent/app/TLK/src/task/medical_segmentation && sh sripts/run_all.sh


###############################################
For that I will be using the following source data configuration:
num_classes:  15
modalities:  {0: 'CT'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [4, 5, 5], 'patch_size': array([ 80, 160, 160]), 'median_patient_size_in_voxels': array([140, 264, 264]), 'current_spacing': array([3.22, 1.62, 1.62]), 'original_spacing': array([3.22, 1.62, 1.62]), 'do_dummy_2D_data_aug': False, 'pool_op_kernel_sizes': [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [1, 2, 2]], 'conv_kernel_sizes': [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]}


I am using source data from this folder:  /home/vmagent/app/dataset/nnUNet_preprocessed/Task508_AMOS_kidney/nnUNetData_plans_v2.1_trgSp_kits19
###############################################
#################

2022-10-14 08:03:19.386149: train loss : 1.0404
2022-10-14 08:05:40.979750: validation loss: 0.0087
2022-10-14 08:05:40.981636: Average global foreground Dice: [0.0]
2022-10-14 08:05:40.981980: (interpret this as an estimate for the Dice of the different classes. This is not exact.)
2022-10-14 08:05:41.279021: lr index 0: 0.001982
2022-10-14 08:05:41.279208: lr index 1: 0.00991
2022-10-14 08:05:41.279391: This epoch took 3509.259070 s

2022-10-14 08:05:41.280476: saving checkpoint...
2022-10-14 08:05:41.633380: done, saving took 0.35 seconds
case_00004 (2, 80, 309, 309)
debug: mirroring True mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: True
data shape: (1, 80, 309, 309)
patch size: [ 80 160 160]
steps (x, y, and z): [[0], [0, 74, 149], [0, 74, 149]]
number of tiles: 9
computing Gaussian
done
prediction done
force_separate_z: None interpolation order: 1
separate z: True lowres axis [0]
separate z, order in z is 0 order inplane is 1
2022-10-14 08:07:06.740799: finished prediction
2022

### 3.3 Visualization of Data and Segmentations
- Download files from server:

   - Images from: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/imagesTr/```

   - Segmentations from: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/labelsTr/```

   - predictions from: ```/home/vmagent/app/dataset/prediction```


- After downloading these files you can visualize them with any volumetric visualization program.
For this we would advise to use [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK)) which already has some great [tutorials](https://www.mitk.org/wiki/Tutorials). 
    - If you have not already downloaded it, here is the [MITK Download Link](https://www.mitk.org/wiki/Downloads)
    
- Here is a demostration of visualization result from MITK on KiTS dataset
![KiTS_visualization](doc/imgs/KiTS_visualization.png)

